In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Filtramos los warnings
import warnings
warnings.filterwarnings("ignore")

# Forzar uso CPU en caso de dispones GPU en el PC
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# 4.2.- Juego Space Invaders de Open AI GYM (con Keras RL)


* GYM de Open AI https://gym.openai.com/: Gym es un conjunto de herramientas para desarrollar y comparar algoritmos de Aprendizaje por Refuerzo (RL). Esta librería ofrece diferentes juegos (entornos) como los juegos de Atari en los que poder probar los algoritmos de RL.


* Keras RL https://keras-rl.readthedocs.io/: keras-rl implementa algunos algoritmos de Aprendizaje por Refuerzo en Python y se integra con con Tensorflow-Keras. Además, keras-rl esta listo para su uso con OpenAI Gym, por lo que resulta sencillo evaluar mediante los juegos de GYM el rendimiento de los Algoritmos.

<hr>

## 1.- Instalación de las librerías (en un Python 3.7)

In [ ]:
!pip3 install tensorflow==2.6.3 --user
!pip3 install keras-rl2 --user
!pip3 install atari_py --user
!pip3 install gym[atari,accept-rom-license]==0.23.0 --user

<hr>

## 2.- Importamos la librería de GYM y atari

In [ ]:
import atari_py
import gym

# Definición de constantes
ENVIRONMENT_NAME = "SpaceInvaders-v0"

<hr>

## 3.- Instanciamos el entorno del Space Invaders y jugamos con un agente que toma acciones aleatorias

In [ ]:
# Instanciamos el Entorno
env = gym.make(ENVIRONMENT_NAME)
# env = gym.make(ENVIRONMENT_NAME, render_mode='human')

# Obtenemos el tamaño de las imagenes del juego y los canales
height, width, n_channels = env.observation_space.shape
print('Tamaño de la imagen: {}x{} - Nº Canales: {}'.format(width, height, n_channels))

# Vemos las posibles acciones del juego:
n_actions = env.action_space.n
actions_info = env.unwrapped.get_action_meanings()
print('\nNº de acciones: {} - Acciones: {}\n'.format(n_actions, actions_info))

# Jugamos 2 partidas de manera aleatoria
episodes = 2
for episode in range(1, episodes+1):
    state = env.reset()  # Inicializamos el entorno
    done = False         # Flag de finalización de la partida (episodio)
    score = 0            # Contador de recompensas
    
    # ! A Jugar ¡ (Hasta que termine la partida -> done == True)
    while not done:
        action = env.action_space.sample()              # Seleccionamos una acción Aleatoria
        n_state, reward, done, info = env.step(action)  # Realizamos la acción aleatoria y obtenemos: 
                                                        # 1. Lista de estados, 2. Recompensa, 3. ¿Fin del juego?, 4. info
        score+=reward                                   # Sumamos la recompensa de la acción
    print('Episode:{} Score:{}'.format(episode, score))
env.close()


<hr>

## 4.- Creamos un modelo de red neuronal convolucional


* Para tomar las acciones vamos a usar una red neuronal convolucional que:

    + Tomará como entrada la imagen de la partida (Alto de 210px, Ancho de 160px y 3 canales)
    + La capa de salida tendrá 6 Neuronas, 1 por posible acción

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

def build_cnn_model():
    model = Sequential()
    model.add(Convolution2D(filters=32, 
                            kernel_size=(8,8), 
                            strides=(4,4), 
                            activation='relu', 
                            input_shape=(3, height, width, n_channels)))
    model.add(Convolution2D(filters=64, 
                            kernel_size=(4,4), 
                            strides=(2,2), 
                            activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(n_actions, activation='linear'))
    
    return model

model = build_cnn_model()
model.summary()

<hr>

## 5.- Construimos el agente con Keral RL y lo entrenamos

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

model = build_cnn_model()
memory = SequentialMemory(limit=1000, window_length=3)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
dqn = DQNAgent(model=model, 
               memory=memory, 
               policy=policy,
               enable_dueling_network=True, 
               nb_actions=n_actions, 
               nb_steps_warmup=100)

# Compilamos el modelo
dqn.compile(Adam(lr=1e-4))

# Entrenamos el modelo
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

<hr>

## 6.- Explotamos el modelo. Nos ponemos a jugar con el agente ya entrenado.

In [ ]:
env_test = gym.make(ENVIRONMENT_NAME, render_mode='human', )
scores = dqn.test(env_test, nb_episodes=5, visualize=False)
print(np.mean(scores.history['episode_reward']))